In [34]:
import pandas as pd
import matplotlib as pyplot
import numpy as np
import sklearn
import math

from hmmlearn import hmm

In [2]:
devices_dataset = pd.read_csv("house_devices.csv")

In [3]:
devices_dataset.head()

,time,lighting2,lighting5,lighting4,refrigerator,microwave
0,1302930703,180,23,195,117,2
1,1302930721,181,23,195,119,2
2,1302930738,180,23,195,117,2
3,1302930765,181,23,195,117,2
4,1302930782,180,23,195,118,2


In [4]:
devices_dataset.shape

(14999, 6)

In [5]:
devices_dataset.describe()

,time,lighting2,lighting5,lighting4,refrigerator,microwave
count,1.499900e+04,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,1.303161e+09,46.444630,17.852990,45.995933,46.283086,6.710247
std,1.315710e+05,91.600463,45.832236,70.079602,66.730693,88.110723
min,1.302931e+09,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.303068e+09,0.000000,1.000000,0.000000,0.000000,2.000000
50%,1.303144e+09,0.000000,1.000000,0.000000,1.000000,2.000000
75%,1.303299e+09,22.000000,21.000000,63.000000,115.000000,2.000000
max,1.303371e+09,362.000000,211.000000,203.000000,916.000000,1798.000000


In [6]:
devices_dataset.nunique()

time            14999
lighting2          58
lighting5          61
lighting4          96
refrigerator      109
microwave          50
dtype: int64

In [20]:
train_size = int(devices_dataset.shape[0] * 0.8)
train_sample = np.random.choice(devices_dataset.index, size=train_size)
train_set  = devices_dataset.loc[train_sample, :]
test_set_sample = devices_dataset.index.difference(train_set.index)
test_set = devices_dataset.loc[test_set_sample, :]

train_set = train_set.sort_values(by="time")
test_set = test_set.sort_values(by="time")

## Learning

In [17]:
models = []
devices_names = devices_dataset.columns[1:]
for n in devices_names:
    hmm_model = hmm.GaussianHMM(n_components=3, n_iter=100)
    X = train_set[[n]]
    hmm_model.fit(X)
    models.append(hmm_model)

In [36]:
results_mtrx = np.zeros((len(devices_names), len(devices_names)))
for i, n in enumerate(devices_names): 
    for j, m in enumerate(models):
        results_mtrx[j, i] = m.score(test_set[[n]])

In [39]:
def select_best_model(X_train, X_test, hidden_states):
    models = []
    scores = []
    for hs in hidden_states:
        hmm_model = hmm.GaussianHMM(n_components=hs, n_iter=100)
        hmm_model.fit(X_train)
        models.append(hmm_model)
        scores.append(hmm_model.score(X_test))
    print(scores)
    prrint(scores.index(max(scores)))
    return models[scores.index(max(scores))]

In [ ]:
test_model = select_best_model(train_set, test_set, [])

In [37]:
pd.DataFrame(results_mtrx)

,0,1,2,3,4
0,-28833.993313,-30780.606470,-45212.872019,-50776.825725,-92608.571811
1,-69118.014414,14891.505885,-50580.050333,-70678.206014,-149854.968158
2,-145.048969,-44865.141371,2559.329280,-26003.027316,-78221.902107
3,-29165.752125,-22962.992708,-30880.044991,-17028.817963,-35498.868069
4,-106200.777174,-45631.705583,-86376.898253,-77230.473885,-24544.446382
